# Установка библиотек

In [2]:
!pip install gitpython


   -------------------------- ------------- 2/3 [gitpython]
   -------------------------- ------------- 2/3 [gitpython]
   ---------------------------------------- 3/3 [gitpython]



In [ ]:
!pip install SurvivalEVAL

In [ ]:
!pip install lifelines

In [1]:
!pip install git+https://github.com/soda-inria/hazardous
!pip install scikit-survival==0.23.0
!pip install scikit-learn==1.4
!pip install torch
!pip install seaborn

  Cloning https://github.com/soda-inria/hazardous to c:\users\pridanova\appdata\local\temp\104\pip-req-build-90m3snk8


  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [3]:
from datetime import datetime
from functools import reduce
# from hazardous import SurvivalBoost
# from hazardous.metrics import integrated_brier_score_survival, integrated_brier_score_incidence
from lifelines import AalenJohansenFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from pycox.evaluation import EvalSurv
from pycox.models import DeepHit
from pycox.preprocessing.label_transforms import LabTransDiscreteTime
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sksurv.ensemble import GradientBoostingSurvivalAnalysis, RandomSurvivalForest
from sksurv.metrics import check_y_survival, CensoringDistributionEstimator
from sksurv.metrics import cumulative_dynamic_auc
from sksurv.util import Surv
from types import NoneType
from typing import List
from zipfile import ZipFile
import copy
import io
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import os
import pandas as pd
import pandas as pd
import re
import seaborn as sns
import torch
import torchtuples as tt
import warnings

# Подготовка данных

In [4]:
warnings.filterwarnings('ignore')

In [8]:
dtypes = {'credit_score': 'Int16', 'first_payment_date': 'str', 'first_time_homebuyer_flag': 'str',
        'maturity_date': 'str', 'MSA': 'Int32', 'MI_%': 'Int16', 'units_numb': 'Int8', 'occupancy_status': 'str',
        'CLTV': 'Int16', 'DTI_ratio': 'Int16', 'orig_UPB': 'Int64', 'LTV': 'Int16', 'orig_interest_rate': 'str',
        'channel': 'str', 'PPM_flag': 'str', 'amortization_type': 'str',
        'property_state': 'str', 'property_type': 'str', 'postal_code': 'Int32', 'id_loan': 'str',
        'loan_purpose': 'str', 'orig_loan_term': 'Int16', 'borrowers_num': 'Int8', 'seller_name': 'str',
        'service_name': 'str', 'super_conf_flag': 'str', 'id_loan_preharp': 'str',
        'program_ind': 'str', 'HARP_ind': 'str', 'property_val_method': 'Int64',
        'int_only_flag': 'str', 'MI_cancel_flag': 'str', 'orig_interest_rate':'float32'}

static = ['credit_score', 'first_time_homebuyer_flag', 'units_numb', 'MSA', 'MI_%', 'occupancy_status', 'CLTV', 'DTI_ratio', 'orig_UPB',
       'LTV', 'orig_interest_rate', 'channel', 'PPM_flag', 'amortization_type',
       'property_state', 'property_type', 'loan_purpose', 'orig_loan_term', 'borrowers_num', 'super_conf_flag',
       'int_only_flag', 'property_val_method']

categ = ['occupancy_status', 'first_time_homebuyer_flag', 'channel', 'PPM_flag', 'amortization_type',
       'property_state', 'borrowers_num', 'int_only_flag', 'property_val_method', 'modification_flag', 'step_mod_flag', 'deferred_payment_plan',
         'ELTV', 'delinq_due_disaster', 'borrowe_asistance_stat_code', 'property_type', 'loan_purpose', 'super_conf_flag']

def get_y(cens, time):
    cens, time = np.array(cens), np.array(time)
    y = np.empty(dtype=[('event', int), ('duration', np.float64)], shape=cens.shape[0])
    y['event'] = cens
    y['duration'] = time
    return y

def get_y_arr(y):
    cens, time = np.array(y.event), np.array(y.duration)
    y = np.empty(dtype=[('event', bool), ('duration', np.float64)], shape=cens.shape[0])
    y['event'] = cens
    y['duration'] = time
    return y


def get_y_arr_int(y):
    cens, time = np.array(y.event), np.array(y.duration)
    y = np.empty(dtype=[('event', int), ('duration', np.float64)], shape=cens.shape[0])
    y['event'] = cens
    y['duration'] = time
    return y


def get_y_event(y_, events: List=[]):
    y = np.empty(dtype=[('event', bool), ('duration', np.float64)], shape=y_.shape[0])
    y['event'] = y_.event.isin(events)
    y['duration'] = y_.duration
    return y

def case1(X_, y_, events: List=[]):
    y, X = y_.copy(), X_.copy()
    mask = y_.event.isin(events)
    y.event = mask.astype('int')
    return X, y

def case2(X_, y_, events: List=[]):
    y, X = y_.copy(), X_.copy()
    mask = y_.event.isin(events)
    y.event = mask
    y = y[mask]
    X = X[mask]
    return X, y

def case3(X_, y_, events: List=[], ):
    y, X = y_.copy(), X_.copy()
    mask = y_.event.isin(events)
    maxm = y_.duration[mask].max()
    y.event = mask
    X = X[y.duration<=maxm]
    y = y[y.duration<=maxm]
    return X, y



def transform_timegrid(curves, time, grid):
    if time.max() < grid.max():
        time = np.hstack([time, np.array([grid.max()+1])])
        if len(curves.shape)==1:
            curves = np.hstack([curves, np.array([0])])
        elif len(curves.shape)==2:
            curves = np.hstack([curves, np.zeros(shape=(curves.shape[0], 1))])
    ind = np.searchsorted(time, grid)
    if len(curves.shape)==1:
      return curves[ind]
    elif len(curves.shape)==2:
      return curves[:, ind]
    else:
      return None


def transform_curves(curves): # noninc
  """
    - curves - current survvial_function(s)
    -------
    Returns:
    - array of nonincreasing survival function(s)
  """
  if len(curves.shape) == 1: curves = curves[None, :]
  return np.array(list(map(lambda tmp:
                           reduce(lambda c, x: (c[0], c[1]+[c[0]])
                           if x > c[0] else (x, c[1]+[x]),
                           tmp[1:], (tmp[0], [tmp[0]]))[1],
                       curves)))

def transform_events(y):
  events = sorted(y.event.unique())
  d = {events[i]:i for i in range(len(events))}
  return y.replace({"event": d}), d

def step_to_array(step_functions):
    shape_=(step_functions.shape[0], step_functions[0].x.shape[0])
    arr = np.empty(shape=shape_)
    for i in range(len(step_functions)):
        arr[i] = step_functions[i].y
    return arr, step_functions[0].x


def str_to_categ(df_col):
    uniq = df_col.unique()
    return df_col.map(dict(zip(uniq, range(len(uniq)))))


class Scaler():
    def __init__(self):
        self.constant_cols=['int_only_flag', 'property_val_method', 'super_conf_flag', 'amortization_type']
        self.categs = list((set(static) & set(categ) - set(self.constant_cols)))
        self.enc = ColumnTransformer(transformers=[('ohe', OneHotEncoder(sparse_output=False).set_output(transform="pandas"), self.categs)], remainder='passthrough')
   
    def fit(self, list_of_df):
        X = pd.concat(list_of_df, axis=0)
        X.drop(self.constant_cols, inplace=True, axis=1)
        self.enc.fit(X)
    
    def transform(self, X):
        X.MSA.fillna(X.MSA.median(), inplace=True)
        X.drop(self.constant_cols, inplace=True, axis=1)
        X = self.enc.transform(X)  
        scaler = StandardScaler().set_output(transform="pandas")
        X = scaler.fit_transform(X)
        return X


def bal280_sample():
    file = r"D:\SurvivalAnalysis\CompetingRisk\result_280bal.csv"
    df = pd.read_csv(file, dtype=dtypes)
    df['event'] = df.zero_balance_code.astype('int')*(df.cens.astype('int'))
    df = df[static+['time', 'event']]
    df = df.apply(lambda x: str_to_categ(x) if x.name in categ else x, axis=0)
    sign = sorted(list(set(df.columns) - {'time', 'event'}))
    y = get_y(df['event'], df['time'] + 1)
    X = df.loc[:, sign]
    return y, X, sign, categ, df

In [69]:
y_large, x_large, _, _, _ = bal280_sample()
sc = Scaler()
sc.fit([x_large])
x_large = sc.transform(x_large)

y_large = pd.DataFrame(y_large)
y_large, dct = transform_events(y_large)
x_train, x_test, y_train, y_test = train_test_split(x_large, y_large, test_size=0.2, stratify=y_large.event, random_state = 1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, stratify=y_train.event, random_state = 1)
TIME_GRID = np.linspace(y_train['duration'].min(), y_train['duration'].max(), 100)

# Модели

## AalenJohansenFitter (lifelines)



In [172]:
def AalenJohansenModel(y_train, y_test):
    events = sorted(y_train.event.unique())
    ibs = np.empty(shape=(len(y_train.event.unique())-1,))
    aj = AalenJohansenFitter()
    for i in events:
      if i>0:
#         print(i)
        T = y_train.duration[y_train.event.isin([i])]
        E = y_train.event[y_train.event.isin([i])]
        aj.fit(T, E, event_of_interest=i)
        curve = 1-aj.cumulative_density_[f'CIF_{i}']
        new_curve = transform_timegrid(curve.values, aj.cumulative_density_.index, TIME_GRID).T.astype(float)

        ibs_self = ibs_remain(
                        get_y_self_event(y_train, [i]),
                        get_y_self_event(y_test, [i]),
                        np.repeat(new_curve[np.newaxis, :], y_test.shape[0], axis=0)[y_test.event == i, :],
                        times=TIME_GRID, axis=-1)
        ibs[i-1] = ibs_self
#         print(ibs_self)
    return ibs
#         print(f"ibs {i:<2}:   {ibs_self:<4.4f}")

## SurvivalBoost (hazardous)

In [177]:
def SurvivalBoostModel(x_train, y_train, x_test, y_test):
    ibs = np.empty(shape=(2, len(y_train.event.unique())-1))
    survival_boost = SurvivalBoost(show_progressbar=False).fit(x_train, y_train)
    cif_sb = survival_boost.predict_cumulative_incidence(x_test, times=TIME_GRID)
    for i in sorted(y_test.event.unique()):
      if i:
        sf = 1-cif_sb[:, i, :]
        ibs_old =  ibs_remain(get_y_self_event(y_train, [i]),
                           get_y_self_event(y_test, [i]), 
                           sf[y_test.event == i, :],
                           times=TIME_GRID, axis=-1)
        ibs[0][i-1] = ibs_old
        sf = transform_curves(sf)
        ibs_noninc =  ibs_remain(get_y_self_event(y_train, [i]),
                           get_y_self_event(y_test, [i]),
                           sf[y_test.event == i, :],
                           times=TIME_GRID, axis=-1)
        ibs[1][i-1] = ibs_noninc
    return ibs

## DeepHit (pycox)

In [6]:
from pycox.models import DeepHit

class DeepHitWrapper:

    class LabTransform(LabTransDiscreteTime):
        def transform(self, durations, events):
            durations, is_event = super().transform(durations, events > 0)
            events[is_event == 0] = 0
            return durations, events.astype('int64')


    class CauseSpecificNet(torch.nn.Module):
        """Network structure similar to the DeepHit paper, but without the residual
        connections (for simplicity).
        """
        def __init__(self, in_features, num_nodes_shared, num_nodes_indiv, num_risks,
                     out_features, batch_norm=True, dropout=None):
            super().__init__()
            self.shared_net = tt.practical.MLPVanilla(
                in_features, num_nodes_shared[:-1], num_nodes_shared[-1],
                batch_norm, dropout,
            )
            self.risk_nets = torch.nn.ModuleList()
            for _ in range(num_risks):
                net = tt.practical.MLPVanilla(
                    num_nodes_shared[-1], num_nodes_indiv, out_features,
                    batch_norm, dropout,
                )
                self.risk_nets.append(net)

        def forward(self, input):
            out = self.shared_net(input)
            out = [net(out) for net in self.risk_nets]
            out = torch.stack(out, dim=1)
            return out
    
    @staticmethod
    def _get_x(df):
        return df.values.astype('float32')

    @staticmethod
    def _get_target(df):
        return df['duration'].values, df['event'].values


    def __init__(self):
        self.labtrans = DeepHitWrapper.LabTransform(100)
        self.best_params_ = {'sigma': 0.3, 'num_nodes_shared': [64, 64],
                            'num_nodes_indiv': [32], 'lr': 0.001, 'dropout': 0.3, 
                            'decoupled_weight_decay': 0.01, 'batch_size': 512,
                            'batch_norm': False, 'alpha': 0.2}
        self.model=None
        self.events = None
        self.TIME_GRID=None

    def fit(self, x_train, y_train, x_val, y_val):
        self.events = y_train.event.unique()
        self.TIME_GRID = np.linspace(y_train['duration'].min(), y_train['duration'].max(), 100)
        
        y_train_dh = self.labtrans.fit_transform(*self._get_target(y_train.sort_values(by='duration')))
        y_val_dh = self.labtrans.transform(*self._get_target(y_val))
        x_train_dh = self._get_x(x_train)
        x_val_dh = self._get_x(x_val)

        val = (x_val_dh, y_val_dh)
        train = (x_train_dh, y_train_dh)
        
        in_features = x_train_dh.shape[1]
        num_nodes_shared = self.best_params_['num_nodes_shared']
        num_nodes_indiv = self.best_params_['num_nodes_indiv']
        num_risks = y_train_dh[1].max()
        out_features = len(self.labtrans.cuts)
        batch_norm = self.best_params_['batch_norm']
        dropout = self.best_params_['dropout']


        net = DeepHitWrapper.CauseSpecificNet(in_features, num_nodes_shared, num_nodes_indiv, num_risks,
                               out_features, batch_norm, dropout)


        optimizer = tt.optim.AdamWR(lr=self.best_params_['lr'],
                                    decoupled_weight_decay=self.best_params_['decoupled_weight_decay'],
                                    cycle_eta_multiplier=0.8)

        self.model = DeepHit(net, optimizer, 
                        alpha=self.best_params_['alpha'], 
                        sigma=self.best_params_['sigma'], 
                        duration_index=self.labtrans.cuts)

        epochs = 512
        batch_size = self.best_params_['batch_size']
        callbacks = [tt.callbacks.EarlyStoppingCycle()]
        verbose = False

        log = self.model.fit(x_train_dh, y_train_dh, batch_size, epochs, callbacks, verbose, val_data=val)
        
        
        
       # cif_dh = model.predict_cif(x_test_dh)
    
    def transform_timegrid(self, curves, time, grid):
        if time.max() < grid.max():
            time = np.hstack([time, np.array([grid.max()+1])])
            if len(curves.shape)==1:
                curves = np.hstack([curves, np.array([0])])
            elif len(curves.shape)==2:
                curves = np.hstack([curves, np.zeros(shape=(curves.shape[0], 1))])
        ind = np.searchsorted(time, grid)
        if len(curves.shape)==1:
            return curves[ind]
        elif len(curves.shape)==2:
            return curves[:, ind]
        else:
            return None
       

    def predict(self, x_test):
        self.events = self.events[self.events > 0]
        x_test_dh = self._get_x(x_test)
        cif = np.moveaxis((1 - self.model.predict_cif(x_test_dh)), (0, 1, 2), (0, 2, 1))
        new = np.zeros(shape=(self.events.shape[0], x_test.shape[0], 100))
        for i in self.events:
            new[i-1] = self.transform_timegrid(cif[i-1], self.model.duration_index, self.TIME_GRID)
        return new


In [5]:
def DeepHitModel(x_train, y_train, x_val, y_val, x_test, y_test):
    ibs = np.empty(shape=(len(y_train.event.unique())-1, ))
    dh = DeepHitWrapper()
    dh.fit(x_train, y_train, x_val, y_val)
    cif = dh.predict(x_test)
    for i in y_train.event.unique():
      if i:
        sf1 = transform_timegrid((1-cif)[i-1], dh.model.duration_index, TIME_GRID)

        ibs_ = ibs_remain(get_y_self_event(y_train, [i]),
                          get_y_self_event(y_test, [i]),
                          sf1[y_test.event == i, :],
                          times=TIME_GRID, axis=-1)
        ibs[i-1] = ibs_
    return ibs
        

## Классы-обертки (OvR, MetaModel)

In [5]:
from typing import List

In [10]:
class OvR:
    def __init__(self, estimator, mode, early_threshold=1.0):
        self.estimator = estimator
        self.mode = mode       # early, all, single
        self.models = None
        self.events = None
        self.TIME_GRID = None
        self.thrsh = early_threshold

    def step_to_array(self, step_functions):
        shape_=(step_functions.shape[0], step_functions[0].x.shape[0])
        arr = np.empty(shape=shape_)
        for i in range(len(step_functions)):
            arr[i] = step_functions[i].y
        return arr, step_functions[0].x

    def transform_timegrid(self, curves, time, grid):
        if time.max() < grid.max():
            time = np.hstack([time, np.array([grid.max()+1])])
            if len(curves.shape)==1:
                curves = np.hstack([curves, np.array([0])])
            elif len(curves.shape)==2:
                curves = np.hstack([curves, np.zeros(shape=(curves.shape[0], 1))])
        ind = np.searchsorted(time, grid)
        if len(curves.shape)==1:
            return curves[ind]
        elif len(curves.shape)==2:
            return curves[:, ind]
        else:
            return None

    def transform_xy(self, X, y, events: List=[], event_of_interest=None):
        if self.mode == 'all':
            X, y = self.case1(X, y, events)
        elif self.mode == 'single':
            X, y = self.case2(X, y, events)
        elif self.mode == 'early':
            X, y = self.case3(X, y, events, early_threshold=self.thrsh)
        elif self.mode == 'mix':
            if event_of_interest in {5, 6}:
                X, y = self.case3(X, y, events, early_threshold=self.thrsh)
            else:
                X, y = self.case2(X, y, events)
        else:
            raise ValueError('Wrong mode')
        return X, y

    def case1(self, X_, y_, events: List=[]):
        y, X = y_.copy(), X_.copy()
        mask = y_.event.isin(events)
        y.event = mask.astype('int')
        return X, y

    def case2(self, X_, y_, events: List=[]):
        y, X = y_.copy(), X_.copy()
        mask = y_.event.isin(events)
        y.event = mask
        y = y[mask]
        X = X[mask]
        y.event = y.event.astype('int')
        return X, y

    def case3(self, X_, y_, events: List=[], early_threshold=1.0):
        y, X = y_.copy(), X_.copy()
        mask = y_.event.isin(events)
        maxm = y_.duration[mask].quantile(early_threshold)
        y.event = mask
        X = X[y.duration<=maxm]
        y = y[y.duration<=maxm]
        y.event = y.event.astype('int')
        return X, y

    def fit(self, X_, y_):
        self.TIME_GRID = np.linspace(y_['duration'].min(), y_['duration'].max(), 100)
        self.model = []
        self.events = y_.event.unique()
        for i in sorted(self.events):
            if i:
                tmp = copy.deepcopy(self.estimator)
                X, y = self.transform_xy(X_, y_, [i], i)
                if type(self.estimator) == CoxPHFitter:
                    X = X.join(y)
                    tmp.fit(X, duration_col='duration', event_col = 'event')
                elif isinstance(self.estimator, AalenJohansenFitter):
                    T, E = y.duration, y.event
                    tmp.fit(T, E, event_of_interest=1)
                else:
                    y = get_y_arr(y)
                    tmp.fit(X, y)
                self.model.append(tmp)

    def predict(self, X_):
        """
        shape (6, 4800, 100)
        
        """
        predictions = np.empty(shape=(len(self.events)-1, X_.shape[0], 100))
        for i in sorted(self.events):
            if i:
                if type(self.estimator) == CoxPHFitter:
                    sf = (self.model[i-1].predict_survival_function(X_, times=TIME_GRID)).T
                elif isinstance(self.estimator, AalenJohansenFitter):
                    sf = 1-self.model[i-1].cumulative_density_[f'CIF_1']
                    sf = transform_timegrid(sf.values, self.model[i-1].cumulative_density_.index, TIME_GRID).T.astype(float)
                    sf = np.repeat(sf[np.newaxis, :], X_.shape[0], axis=0)
                else:
                    sf = self.model[i-1].predict_survival_function(X_)
                    sf, times = step_to_array(sf)
                    sf = transform_timegrid(sf, times, TIME_GRID)
                predictions[i-1]=sf
        return predictions



In [11]:
# class SurvivalBoost:
    # pass

In [12]:
class MetaModel:
    def __init__(self, estimator, mode='weighted'):
        self.ovr = estimator
        self.meta_model = RandomForestClassifier(n_jobs=-1, random_state=42)
        self.mode = mode

    def fit(self, X_, y_, X_val=None, y_val=None):
        if (not (X_val is None)): self.ovr.fit(X_, y_, X_val, y_val)
        else: self.ovr.fit(X_, y_)
        mask = y_.event>0
        self.meta_model.fit(X_[mask], y_.event[mask])

    def predict(self, X_):
        all_preds = self.ovr.predict(X_) # (7, 4800, 100)
        if self.mode == 'best':
            events_pred = self.meta_model.predict(X_)
            selected_preds = np.zeros((X_.shape[0], 100)) # (4800, 100)
            for i in range(X_.shape[0]):
                selected_preds[i] = all_preds[events_pred[i]-1, i, :]
            return selected_preds
        elif self.mode == 'weighted':
            events_pred = self.meta_model.predict_proba(X_)
            tmp = np.moveaxis(all_preds, (0, 1, 2), (1, 0, 2))
            return np.sum(tmp * events_pred[..., np.newaxis], axis=1)

In [17]:
def get_y_self_event(y_, events: List=[]):
    l = sum(y_.event.isin(events))
    y = np.empty(dtype=[('event', bool), ('duration', np.float64)], shape=l)
    y["duration"] = y_[y_.event.isin(events)]["duration"]
    y["event"] = 1
    return y

# Метрики

### IBS competrisk

In [337]:

# новая версия (исправлено 1/(K+1))
def ibs_remain_crk(survival_train, survival_test, estimate, times, axis=-1):
    test_event, test_time = survival_test['event'], survival_test['duration']
    estimate = np.array(estimate)
    estimate = np.moveaxis(estimate, (0, 1, 2), (1, 0, 2))
    times = np.array(times)
    n_samples = estimate.shape[0]
    K = estimate.shape[1]
    n_times = estimate.shape[2]
    estimate[estimate == -np.inf] = 0
    estimate[estimate == np.inf] = 0
    
    before = times[np.newaxis, :] < test_time[:, np.newaxis]
    after_event = np.zeros((n_samples, K, n_times), dtype=bool)
    
    for k in range(K):
        after_event[:, k, :] = (~before) & ((test_event == k+1)[:, np.newaxis])
    
    estim_before_sq = (1 - estimate) ** 2
    estim_after_sq = estimate ** 2
    
    brier_scores = np.where(
        before[:, np.newaxis, :],
        estim_before_sq,
        np.where(after_event, estim_after_sq, 0))
    
    brier_scores_n = np.where(
        before,
        np.sum(brier_scores, axis=1) / (K + 1),
        np.sum(brier_scores, axis=1))
    
    N = np.sum(np.array([np.where(test_time < t, test_event, 1) for i, t in enumerate(times)]), axis=1)
    brier_scores = np.sum(brier_scores_n, axis=0)
    brier_scores = np.where(N > 0, brier_scores / N, 0)
 
    time_diff = times[-1] - times[0] if times[-1] > times[0] else 1
    
    if axis == -1: 
        return np.trapz(brier_scores, times) / time_diff
    elif axis == 0: 
        return np.trapz(brier_scores_n, times, axis=1) / time_diff
    elif axis == 1:
        return brier_scores
    
    return None

### IBS competrisk balanced

In [1]:
def ibs_remain_crk_balanced(survival_train, survival_test, estimate, times, axis=-1):
    test_event, test_time = survival_test['event'], survival_test['duration']
    estimate = np.array(estimate)
    estimate = np.moveaxis(estimate, (0, 1, 2), (1, 0, 2))
    times = np.array(times)
    
    n_samples = estimate.shape[0]
    K = estimate.shape[1]
    n_times = estimate.shape[2]
    
    estimate[estimate == -np.inf] = 0
    estimate[estimate == np.inf] = 0
    
    before = times[np.newaxis, :] < test_time[:, np.newaxis]
    
    estim_before_sq = (1 - estimate) ** 2 
    estim_after_sq = estimate ** 2

    delta_indices = test_event.astype(int) - 1  # (n_samples,) : shift 1->0, 2->1, etc.
    valid_event = test_event > 0
    vals = np.zeros((n_samples, n_times))
    vals[valid_event] = estim_before_sq[
        np.arange(n_samples)[valid_event, np.newaxis], 
        delta_indices[valid_event, np.newaxis], 
        np.arange(n_times)
    ]
    
    all_sum = np.sum(estim_before_sq, axis=1)  # (n_samples, n_times)
    other_types = np.where(valid_event[:, np.newaxis], 
                               all_sum - vals,
                               all_sum)
    bs_before = 0.5 * vals + 0.5 * other_types / K
    vals_after = np.zeros((n_samples, n_times))

    vals_after[valid_event] = estim_after_sq[
        np.arange(n_samples)[valid_event, np.newaxis], 
        delta_indices[valid_event, np.newaxis], 
        np.arange(n_times)
    ]
    
    bs_after = vals_after * (~before) * (valid_event > 0)[:, np.newaxis]
    bs_obs = np.where(before, bs_before, bs_after)
    
    N = np.sum(np.array([np.where(test_time < t, test_event, 1)
                         for i, t in enumerate(times)]), axis=1)
    
    brier_scores = np.sum(bs_obs, axis=0)
    brier_scores = np.where(N > 0, brier_scores / N, 0)
    
    time_diff = times[-1] - times[0] if times[-1] > times[0] else 1
    
    if axis == -1: 
        return np.trapz(brier_scores, times) / time_diff
    elif axis == 0: 
        return np.trapz(bs_obs, times, axis=1) / time_diff
    elif axis == 1:
        return brier_scores
    
    return None

### IBS

In [20]:
def ibs_remain(survival_train, survival_test, estimate, times, axis=-1):
    """ IBS with equal impact of partial observation with controlled quantity """
    test_event, test_time = check_y_survival(survival_test, allow_all_censored=True)
    estimate = np.array(estimate)
    if estimate.ndim == 1 and times.shape[0] == 1:
        estimate = estimate.reshape(-1, 1)
    estimate[estimate == -np.inf] = 0
    estimate[estimate == np.inf] = 0

    estim_before = np.square(estimate) * test_event[np.newaxis, :].T
    estim_after = np.square(1 - estimate)
    brier_scores = np.array([np.where(test_time < t,
                                      estim_before[:, i],
                                      estim_after[:, i])
                             for i, t in enumerate(times)])
    N = np.sum(np.array([np.where(test_time < t, test_event, 1)
                         for i, t in enumerate(times)]), axis=1)
    time_diff = times[-1] - times[0] if times[-1] > times[0] else 1
    if axis == -1:  # mean ibs for each time and observation
        # brier_scores = np.mean(brier_scores, axis=1)
        brier_scores = np.where(N > 0, 1 / N, 0) * np.sum(brier_scores, axis=1)
        return np.trapz(brier_scores, times) / time_diff
    elif axis == 0:  # ibs for each observation
        return np.trapz(brier_scores, times, axis=0) / time_diff
    elif axis == 1:  # bs in time (for graphics)
        # brier_scores = np.mean(brier_scores, axis=1)
        brier_scores = np.where(N > 0, 1 / N, 0) * np.sum(brier_scores, axis=1)
        return brier_scores
    return None

### AUPRC

In [11]:
def calculate_survival_auprc_vectorized(y_true, preds, time_points, num_phi_steps=100):
   
    n_obs = preds.shape[0]
    ext_time_points = np.concatenate([[0], time_points])
    ext_probs = np.concatenate([np.ones((n_obs, 1)), preds], axis=1)
    ext_probs = np.minimum.accumulate(ext_probs, axis=1)
    true_times = y_true['duration'].reshape(-1, 1)
    is_event = y_true['event'].astype(bool).reshape(-1, 1)
    true_times[true_times == 0] = 1e-8
    phi_grid = np.linspace(0, 1, num_phi_steps).reshape(1, -1)
    t_early_matrix = true_times * phi_grid
    phi_grid_safe = np.copy(phi_grid)
    phi_grid_safe[phi_grid_safe == 0] = 1e-8
    t_late_matrix = true_times / phi_grid_safe
    t_late_matrix[:, phi_grid[0] == 0] = np.inf
    
    indices_early = np.searchsorted(ext_time_points, t_early_matrix, side='right') - 1
    indices_late = np.searchsorted(ext_time_points, t_late_matrix, side='right') - 1
    row_indexer = np.arange(n_obs).reshape(-1, 1)

    s_early = ext_probs[row_indexer, indices_early]
    s_late = ext_probs[row_indexer, indices_late]
    
    integral_values = np.where(is_event, s_early - s_late, s_early)
    
    auprc_scores = np.sum(integral_values, axis=1) / num_phi_steps
    
    return np.mean(auprc_scores)

# Эксперименты

### OVR

In [ ]:
early_threshold = 0.99
col = [f'тип {i}' for i in [1, 2, 3, 9, 15, 16, 96]]
ind = [f'OVR({i}) - {j}' for i in ['Cox', 'RSF'] for j in ['single', 'all', 'early']]
ovrs = [OvR(CoxPHFitter(penalizer=0.02), mode='single', early_threshold=early_threshold),
        OvR(CoxPHFitter(penalizer=0.02), mode='all', early_threshold=early_threshold),
        OvR(CoxPHFitter(penalizer=0.02), mode='early', early_threshold=early_threshold),

        OvR(RandomSurvivalForest(n_jobs=-1, random_state=42), mode='single', early_threshold=early_threshold),
        OvR(RandomSurvivalForest(n_jobs=-1, random_state=42), mode='all', early_threshold=early_threshold),
        OvR(RandomSurvivalForest(n_jobs=-1, random_state=42), mode='early', early_threshold=early_threshold)]

for elem in ovrs:
    elem.fit(x_train, y_train)

preds = [model.predict(x_test) for model in ovrs]
ovr_ibs = np.empty(shape=(7, len(preds)))
ovr_auc = np.empty(shape=(7, len(preds)))


for i in y_train.event.unique():
  if i:
    for j, cif in enumerate(preds):
        sf = cif[i-1, ...]    
        ovr_auc[i-1][j] = calculate_survival_auprc_vectorized(
            get_y_self_event(y_test, [i]), 
            sf[y_test.event == i, :],
            TIME_GRID)
        ovr_ibs[i-1][j] = ibs_remain(get_y_self_event(y_train, [i]),
            get_y_self_event(y_test, [i]),
            sf[y_test.event == i, :],
            times=TIME_GRID, axis=-1)

df_auc = pd.DataFrame(ovr_auc.T, columns=col, index=ind)
df_ibs = pd.DataFrame(ovr_ibs.T, columns=col, index=ind)


In [ ]:
df_ibs

In [ ]:
df_auc

### Meta best

In [ ]:
early_threshold = 0.99
col = [f'тип {i}' for i in [1, 2, 3, 9, 15, 16, 96]]
ind = [f'Meta_w(OVR({i})) - {j}' for i in ['Cox', 'RSF'] for j in ['single', 'all', 'early']]
ovrs = [OvR(CoxPHFitter(penalizer=0.02), mode='single', early_threshold=early_threshold),
        OvR(CoxPHFitter(penalizer=0.02), mode='all', early_threshold=early_threshold),
        OvR(CoxPHFitter(penalizer=0.02), mode='early', early_threshold=early_threshold),

        OvR(RandomSurvivalForest(n_jobs=-1, random_state=42), mode='single', early_threshold=early_threshold),
        OvR(RandomSurvivalForest(n_jobs=-1, random_state=42), mode='all', early_threshold=early_threshold),
        OvR(RandomSurvivalForest(n_jobs=-1, random_state=42), mode='early', early_threshold=early_threshold)]

metas_b = list(map(lambda x: MetaModel(x, mode='best'), ovrs))
for m in metas_w:
    m.fit(x_train, y_train)
    
preds = [m.predict(x_test) for m in metas_b]
meta_b_ibs = np.empty(shape=(7, len(preds)))
meta_b_auc = np.empty(shape=(7, len(preds)))

for i in y_train.event.unique():
  if i:
    for j, cif in enumerate(preds):
        # sf = cif[i-1, ...]    
        meta_b_auc[i-1][j] = calculate_survival_auprc_vectorized(
            get_y_self_event(y_test, [i]), 
            cif[y_test.event == i, :],
            TIME_GRID)
        meta_b_ibs[i-1][j] = ibs_remain(get_y_self_event(y_train, [i]),
            get_y_self_event(y_test, [i]),
            cif[y_test.event == i, :],
            times=TIME_GRID, axis=-1)

df_ibs = pd.DataFrame(meta_b_ibs.T, columns=col, index=ind)
df_auc = pd.DataFrame(meta_b_auc.T, columns=col, index=ind)


In [ ]:
df_ibs

In [ ]:
df_auc

### Meta weighted

In [ ]:
early_threshold = 0.99
col = [f'тип {i}' for i in [1, 2, 3, 9, 15, 16, 96]]
ind = [f'Meta_w(OVR({i})) - {j}' for i in ['Cox', 'RSF'] for j in ['single', 'all', 'early']]
ovrs = [OvR(CoxPHFitter(penalizer=0.02), mode='single', early_threshold=early_threshold),
        OvR(CoxPHFitter(penalizer=0.02), mode='all', early_threshold=early_threshold),
        OvR(CoxPHFitter(penalizer=0.02), mode='early', early_threshold=early_threshold),

        OvR(RandomSurvivalForest(n_jobs=-1, random_state=42), mode='single', early_threshold=early_threshold),
        OvR(RandomSurvivalForest(n_jobs=-1, random_state=42), mode='all', early_threshold=early_threshold),
        OvR(RandomSurvivalForest(n_jobs=-1, random_state=42), mode='early', early_threshold=early_threshold)]

metas_w = list(map(lambda x: MetaModel(x, mode='weighted'), ovrs))
for m in metas_w:
    m.fit(x_train, y_train)
    
preds = [m.predict(x_test) for m in metas_w]
meta_w_ibs = np.empty(shape=(7, len(preds)))
meta_w_auc = np.empty(shape=(7, len(preds)))

for i in y_train.event.unique():
  if i:
    for j, cif in enumerate(preds):
        # sf = cif[i-1, ...]    
        meta_w_auc[i-1][j] = calculate_survival_auprc_vectorized(
            get_y_self_event(y_test, [i]), 
            cif[y_test.event == i, :],
            TIME_GRID)
        meta_w_ibs[i-1][j] = ibs_remain(get_y_self_event(y_train, [i]),
                          get_y_self_event(y_test, [i]),
                          cif[y_test.event == i, :],
                          times=TIME_GRID, axis=-1)

df_ibs = pd.DataFrame(meta_w_ibs.T, columns=col, index=ind)
df_auc = pd.DataFrame(meta_w_auc.T, columns=col, index=ind)


In [ ]:
df_ibs

In [ ]:
df_auc